# Equação de Tolman-Oppenheimer-Volkoff (TOV)

Partimos da métrica de Swarzschild (trabalhamos com unidades onde $c = G = 1$):

$$ds^2 =  -\left( 1 - \frac{2M}{r} \right) dt^2 + {\left( 1 - \frac{2M}{r} \right)}^{-1} dr^2 + r^2 d\theta^2 + r^2 \sin^2 \theta d\phi^2$$

e do tensor de energia momentum para um fluido perfeito:

$$T^{\mu \nu} = (\rho + p)U^{\mu} U^{\nu} + p g^{\mu \nu} $$

As equações de campo de Einsten nos dão a relação entre a métrica e o tensor de energia momentum:

$$R_{\mu \nu} - \frac{1}{2} R g_{\mu \nu} = 8 \pi T_{\mu \nu}$$

Com isso podemos igualar ambas essas grandezas e, após um pouco de álgebra, chegamos a:

$$\frac{dp}{dr} = -\frac{m(r) \epsilon (r)}{r^2} \left[ 1 - \frac{p(r)}{\epsilon(r)} \right] \left[ 1 + \frac{4 \pi r^3 p(r)}{m(r)} \right] {\left[ 1 - \frac{2m(r)}{r} \right]}^{-1}$$
$$\frac{dm}{dr} = 4 \pi r^2 \epsilon(r)$$


São duas equações diferenciais acopladas, que, com as condições de que a massa no centro seja $m(0) = 0$ e que a pressão na superfície seja $p(r = R) = 0$ onde $R$ ficará definido como o raio da estrela e $m(r = R) = M$ é a massa total da estrela (massa gravitacional). Ainda precisamos de uma condição inicial na pressão que será $p(0) = p_0$, essa condição porém será arbitrária. *Estrelas com $p_0$ diferente terão valores de massa ($M$) e raio ($R$) diferentes*. Ainda nos falta uma informação, que é $\epsilon(r)$.

# Equação de estado

É a equação de estado que nos diz sobre como a matéria da estrela é constituida e como ela interage. Com uma equação de estado podemos encontrar uma relação entre $p(r)$ e $\epsilon(r)$ e finalmente resolvermos o sistema. A equação de estado mais simples que existe é a equação de estado politrópica onte temos:

$$p = K\epsilon^{\gamma}$$

onde $K$ é uma constante e $\gamma$ um índice politrópico (não lembro ao certo se o $\gamma$ é o índice politrópico). Porém, essa equação de estado simplesmente considera a estrela como uma bola de gás e podemos achar bem difícil e não tão agradável trabalhar com equações de estado politrópicas.

# Método de Runge-Kutta

O método mais clássico de resolver equações diferenciais é o de Newton (às vezes também chamado de Euler) onde seguimos a reta tangente (que é obtida da equação diferencial) e vamos seguindo ela para obtermos as curvas das soluções (geralmente descrevem como se estivéssemos integrando). Quando nos deparamos com um problema não-linear esse método acaba ficando menos preciso e altamente dependente de passos muito pequenos. Então precisamos de um método que funcione também para problemas não-lineares (que é o caso da equação de TOV), o método que utilizaremos é o de Runge-Kutta, em específico o RK4.

Um problema de valor inicial é dado por:

$$\frac{dy}{dx} = f(t, y)$$
$$y(t_0) = y_0$$

Partindo de um $y(t_0) = y_0$ e com um passo $h$ temos o próximo valor da solução:

$$y_{n+1} = y_{n} + h \left( \frac{k_{n1} + 2k_{n2} + 2k_{n3} + k_{n4}}{6} \right)$$
onde
$$k_{n1} = f(t_n, y_n)$$
$$k_{n2} = f \bigg( t_n + \frac{h}{2}, h + \frac{k_{n1}}{2} \bigg)$$
$$k_{n3} = f \bigg( t_n + \frac{h}{2}, h + \frac{k_{n2}}{2} \bigg)$$
$$k_{n4} = f(t_n + h, y_n + hk_{n3})$$

Quando temos sistemas de equações diferenciais basta tratarmos $y$ como um vetor. Vamos chamar agora as soluções de $\vec{x}$ que satisfazem o problema de valor inicial:

$$\frac{d\vec{x}}{dt} = \vec{f}(t, \vec{x})$$
$$\vec{x}(t_0) = \vec{x}_0$$

e então teremos a seguinte forma:

$$\vec{x}_{n+1} = \vec{x}_{n} + h \left( \frac{\vec{k}_{n1} + 2\vec{k}_{n2} + 2\vec{k}_{n3} + \vec{k}_{n4}}{6} \right)$$
onde
$$\vec{k}_{n1} = \vec{f}(t_n, \vec{x}_n)$$
$$\vec{k}_{n2} = \vec{f} \bigg( t_n + \frac{h}{2}, \vec{x}_{n} + \frac{\vec{k}_{n1}}{2} \bigg)$$
$$\vec{k}_{n3} = \vec{f} \bigg( t_n + \frac{h}{2}, \vec{x}_{n} + \frac{\vec{k}_{n2}}{2} \bigg)$$
$$\vec{k}_{n4} = \vec{f}(t_n + h, \vec{x}_n + h\vec{k}_{n3})$$